In [1]:
import pandas as pd
import numpy as np
import os
from copy import copy
from tqdm import tqdm
from glob import glob
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras

from rdkit.Chem import MolStandardize, MolFromSmiles, MolToSmiles

from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split


2022-11-15 21:22:53.704715: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 21:22:53.858719: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
with open("../data/raw/ZINC_results.smi") as f:
    smiles = [s.split("\t")[0].rstrip() for s in f]
smiles[:4]


['c1ccc(cc1)[N-]S(=O)(=O)CCCCCCC(=O)NO',
 'c1cc(ccc1[N+](=O)[O-])OC[C@H]2CO2',
 'c1cc(ccc1[N+](=O)[O-])OC[C@@H]2CO2',
 'c1cc(c(cc1[N+](=O)[O-])[N+](=O)[O-])Cl']

In [3]:
class SmilesTokenizer(object):
    def __init__(self):
        atoms = [
            "Al",
            "As",
            "B",
            "Br",
            "C",
            "Cl",
            "F",
            "H",
            "I",
            "K",
            "Li",
            "N",
            "Na",
            "O",
            "P",
            "S",
            "Se",
            "Si",
            "Te",
        ]
        special = [
            "(",
            ")",
            "[",
            "]",
            "=",
            "#",
            "%",
            "0",
            "1",
            "2",
            "3",
            "4",
            "5",
            "6",
            "7",
            "8",
            "9",
            "+",
            "-",
            "se",
            "te",
            "c",
            "n",
            "o",
            "s",
        ]
        padding = ["G", "A", "E"]

        self.table = sorted(atoms, key=len, reverse=True) + special + padding
        table_len = len(self.table)

        self.table_2_chars = list(filter(lambda x: len(x) == 2, self.table))
        self.table_1_chars = list(filter(lambda x: len(x) == 1, self.table))

        self.one_hot_dict = {}
        for i, symbol in enumerate(self.table):
            vec = np.zeros(table_len, dtype=np.float32)
            vec[i] = 1
            self.one_hot_dict[symbol] = vec

    def tokenize(self, smiles):
        smiles = smiles + " "
        N = len(smiles)
        token = []
        i = 0
        while i < N:
            c1 = smiles[i]
            c2 = smiles[i : i + 2]

            if c2 in self.table_2_chars:
                token.append(c2)
                i += 2
                continue

            if c1 in self.table_1_chars:
                token.append(c1)
                i += 1
                continue

            i += 1

        return token

    def one_hot_encode(self, tokenized_smiles):
        result = np.array(
            [self.one_hot_dict[symbol] for symbol in tokenized_smiles], dtype=np.float32
        )
        result = result.reshape(1, result.shape[0], result.shape[1])
        return result


In [4]:
class Preprocessor(object):
    def __init__(self):
        self.normarizer = MolStandardize.normalize.Normalizer()
        self.lfc = MolStandardize.fragment.LargestFragmentChooser()
        self.uc = MolStandardize.charge.Uncharger()

    def process(self, smi):
        mol = MolFromSmiles(smi)
        if mol:
            mol = self.normarizer.normalize(mol)
            mol = self.lfc.choose(mol)
            mol = self.uc.uncharge(mol)
            smi = MolToSmiles(mol, isomericSmiles=False, canonical=True)
            return smi
        else:
            return None


In [5]:
pp = Preprocessor()

print(f"input SMILES num: {len(smiles)}")
print("start preprocessing...")

smiles = [pp.process(smi) for smi in tqdm(smiles)]
smiles = list(set([s for s in smiles if s]))

# token limits (34 to 74)
st = SmilesTokenizer()
smiles_tokenized = [st.tokenize(smi) for smi in tqdm(smiles)]
smiles_processed = []
smiles_max_len = 0


# check for not recognized tokens
err = 0
err_tokens = []
for i in range(len(smiles)):
    if smiles[i] != "".join(smiles_tokenized[i]):
        print("=====================================")
        print(len(smiles[i]), " :", smiles[i])
        print(len(smiles_tokenized[i]), " :" ,smiles_tokenized[i])
        for char in smiles[i]:
            if char not in smiles_tokenized[i]:
                err_tokens.append(char)
        err += 1
print("Error: ", err)
print("Error Tokens: ", err_tokens)


for tokenized in smiles_tokenized:
    if 34 <= len(tokenized) <= 74:
        smiles_processed.append(tokenized)
        # update smiles max len
        if len(tokenized) > smiles_max_len:
            smiles_max_len = len(tokenized)

print(f"output SMILES num: {len(smiles_processed)}")


input SMILES num: 5000
start preprocessing...


100%|██████████| 4998/4998 [00:00<00:00, 47463.98it/s]

Error:  0
Error Tokens:  []
output SMILES num: 4332


In [6]:
def _pad(tokenized_smi):
    return (
        ["G"] + tokenized_smi + ["E"] + ["A" for _ in range(smiles_max_len - len(tokenized_smi))]
    )

def _padding(data):
    padded_smiles = [_pad(t_smi) for t_smi in data]
    return padded_smiles


In [7]:
# add paddings
print("".join(smiles_processed[0]))
smiles_processed = _padding(smiles_processed)
print("".join(smiles_processed[0]))


CN1CC2CCC1CN(C(=O)Nc1ccc(Cl)cc1Cl)C2
GCN1CC2CCC1CN(C(=O)Nc1ccc(Cl)cc1Cl)C2EAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


In [8]:
# one hot encode
x, y = [], []

for tp_smi in smiles_processed[:5]:
    print("-----------------------------------")
    print("".join(tp_smi[:-1]))
    print("".join(tp_smi[1:]))

for tp_smi in smiles_processed:
    _x = [st.one_hot_dict[symbol] for symbol in tp_smi[:-1]]
    x.append(_x)
    _y = [st.one_hot_dict[symbol] for symbol in tp_smi[1:]]
    y.append(_y)

x = np.array(x, dtype=np.float32)
y = np.array(y, dtype=np.float32)
print(x.shape)
x


-----------------------------------
GCN1CC2CCC1CN(C(=O)Nc1ccc(Cl)cc1Cl)C2EAAAAAAAAAAAAAAAAAAAAAAAAAAAA
CN1CC2CCC1CN(C(=O)Nc1ccc(Cl)cc1Cl)C2EAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
-----------------------------------
GCCn1ccc2ccc(CN3CC4CCC3CN(S(=O)(=O)N(C)C)C4)cc21EAAAAAAAAAAAAAAA
CCn1ccc2ccc(CN3CC4CCC3CN(S(=O)(=O)N(C)C)C4)cc21EAAAAAAAAAAAAAAAA
-----------------------------------
GCCS(=O)(=O)N1CC2CCC1CN(C(=O)c1ccncc1)C2EAAAAAAAAAAAAAAAAAAAAAAA
CCS(=O)(=O)N1CC2CCC1CN(C(=O)c1ccncc1)C2EAAAAAAAAAAAAAAAAAAAAAAAA
-----------------------------------
GO=C(c1cccnc1-c1ccc(OC(F)(F)F)cc1)N1CC2CCC1C2EAAAAAAAAAAAAAAAAAA
O=C(c1cccnc1-c1ccc(OC(F)(F)F)cc1)N1CC2CCC1C2EAAAAAAAAAAAAAAAAAAA
-----------------------------------
GCc1cc(CN2CC3CCC2CN(C2Cc4ccccc4C2)C3)on1EAAAAAAAAAAAAAAAAAAAAAAA
Cc1cc(CN2CC3CCC2CN(C2Cc4ccccc4C2)C3)on1EAAAAAAAAAAAAAAAAAAAAAAAA
(4332, 64, 47)


array([[[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       ...,

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1.

In [9]:
# pick 20% for validation
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)


In [10]:
for i in range(x.shape[0]):
    if x[i].shape != y[i].shape:
        print(x[i].shape)
        print(y[i].shape)


In [11]:
# model

np.set_printoptions(precision=3, suppress=True)
tfd = tfp.distributions


class CustomizedLayer_Polarizer(keras.layers.Layer):
    def __init__(self, units=32):
        super(CustomizedLayer_Polarizer, self).__init__()

    def call(self, inputs):
        G_thesis = inputs
        G_antithesis = 1 - inputs

        return [G_thesis, G_antithesis]


class CustomizedLayer_Attention(keras.layers.Layer):
    def __init__(self, units=32):
        super(CustomizedLayer_Attention, self).__init__()

    def call(self, inputs):
        # G_LSTM= inputs[:,:60]
        # G_Attention= inputs[:,60:]
        # res= tf.math.add(G_LSTM, G_Attention)
        elem_prod = inputs[:, :, :60] + inputs[:, :, 60:]
        # res = k.sum(elem_prod, axis=-1, keepdims=True)
        return elem_prod


def prior_mean_field(kernel_size, bias_size, dtype=None):  # prior Function
    n = kernel_size + bias_size
    return lambda t: tfd.Independent(
        tfd.Normal(loc=tf.zeros(n, dtype=dtype), scale=tf.ones(n)), reinterpreted_batch_ndims=1
    )


def posterior_mean_field(kernel_size, bias_size=0, dtype=None):  # Posterior Function
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.0))
    return tf.keras.Sequential(
        [
            tfp.layers.VariableLayer(2 * n, dtype=dtype),
            tfp.layers.DistributionLambda(
                lambda t: tfd.Independent(
                    tfd.Normal(loc=t[..., :n], scale=1e-5 + 0.01 * tf.nn.softplus(c + t[..., n:])),
                    reinterpreted_batch_ndims=1,
                )
            ),
        ]
    )


data_len = x.shape[0]
hidden_units = [70, 70, 70]
batch_size = 50
counter_L = 0
look_back = 1
model = Sequential()
# InData_Ex1 = layers.Input(shape=([1, 70]), name="Input_Ex1")
# InData_Ex2 = layers.Input(shape=([1, 70]), name="polarizer")
# InData_Ex3 = layers.Input(shape=([1, 70]), name="Input_EX3")
InData_Ex1 = layers.Input(shape=([64, 47]), name="Input_Ex1")
InData_Ex2 = layers.Input(shape=([64, 47]), name="polarizer")
InData_Ex3 = layers.Input(shape=([64, 47]), name="Input_EX3")
EX_lstm1 = layers.LSTM(60, return_sequences=True)(InData_Ex1)
EX_lstm2 = layers.LSTM(60, return_sequences=True)(InData_Ex2)
GateIn = layers.Dense(units=60, activation="sigmoid")(InData_Ex3)
Gate_pp = layers.Dense(units=60, activation="sigmoid")(GateIn)
Gate_pp = layers.Dense(units=60, activation="sigmoid")(Gate_pp)
CFPG = CustomizedLayer_Polarizer(units=60)(Gate_pp)
# GatesODD = layers.Dense(units=60, activation='sigmoid')(GatesIn)
MultiplictionEven_In = layers.Concatenate(axis=-1)([EX_lstm1, CFPG[0]])
MultiplictionEven_In = CustomizedLayer_Attention()(MultiplictionEven_In)
EX_lstm1 = layers.LSTM(60, return_sequences=True)(MultiplictionEven_In)
MultiplictionEven = layers.Dense(units=35, activation="sigmoid")(EX_lstm1)
MultiplictionODD_In = layers.Concatenate(axis=-1)([EX_lstm2, CFPG[1]])
MultiplictionODD_In = CustomizedLayer_Attention()(MultiplictionODD_In)
EX_lstm2 = layers.LSTM(60, return_sequences=True)(MultiplictionODD_In)
MultiplictionODD = layers.Dense(units=35, activation="sigmoid")(EX_lstm2)
# features = layers.Concatenate([InData_Ex1, InData_Ex2, InData_Ex3, InData_Ex4])
InData = layers.Concatenate(axis=-1)([MultiplictionEven, MultiplictionODD])
InData = layers.BatchNormalization()(InData)
features = InData
for units in hidden_units:
    features = tfp.layers.DenseVariational(
        units=units,
        make_prior_fn=prior_mean_field,
        make_posterior_fn=posterior_mean_field,
        kl_weight=1 / data_len,
        activation="relu",
    )(features)
# features = layers.Dense(units=70, activation="sigmoid")(features)
features = layers.Dense(units=47, activation="sigmoid")(features)
model = keras.Model(inputs=[InData_Ex1, InData_Ex2, InData_Ex3], outputs=features)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.summary()


2022-11-15 21:23:02.267064: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_EX3 (InputLayer)         [(None, 64, 47)]     0           []                               
                                                                                                  
 dense (Dense)                  (None, 64, 60)       2880        ['Input_EX3[0][0]']              
                                                                                                  
 dense_1 (Dense)                (None, 64, 60)       3660        ['dense[0][0]']                  
                                                                                                  
 Input_Ex1 (InputLayer)         [(None, 64, 47)]     0           []                               
                                                                                              

In [21]:
callbacks = []
callbacks.append(
    ModelCheckpoint(
        filepath=os.path.join(
            "../reports/",
            "2022-11-15",
            "test",
            "checkpoints",
            '{epoch:02d}.hdf5'),
        monitor="val_loss",
        mode="min",
        save_best_only=False,
        save_weights_only=True,
        verbose=True,
    )
)
# create checkpoints dir
os.makedirs(os.path.join(
    "../reports/",
    "2022-11-15",
    "test",
    "checkpoints"), exist_ok=True)

callbacks.append(
    TensorBoard(
        log_dir=os.path.join(
            "../reports/",
            "2022-11-15",
            "test",
            "logs",
        ),
        write_graph=True,
        write_images=True,
        update_freq="epoch",
    )
)


In [22]:
history = model.fit(
            {"Input_Ex1": x_train, "polarizer": x_train, "Input_EX3": x_train},
            y_train,
            # steps_per_epoch=x_train.shape[0],
            steps_per_epoch=100,
            epochs=5,
            verbose=True,
            validation_data=({"Input_Ex1": x_val, "polarizer": x_val, "Input_EX3": x_val}, y_val),
            # validation_steps=x_val.shape[0],
            validation_steps=30,
            use_multiprocessing=True,
            shuffle=True,
            callbacks=callbacks,
        )

# last_weight_file = glob(
#     os.path.join(
#         "../reports/",
#         "2022-11-15",
#         "test",
#         "checkpoints",
#         'test-{30:02}*.hdf5')
# )[0]


Layer CustomizedLayer_Polarizer has arguments ['units']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config
Epoch 1/5
100/100 [==============================] - ETA: 0s - loss: 8.8754 - accuracy: 0.7227
Epoch 1: saving model to ../reports/2022-11-15/test/checkpoints/01.hdf5
100/100 [==============================] - 10s 96ms/step - loss: 8.8754 - accuracy: 0.7227
Epoch 2/5
100/100 [==============================] - ETA: 0s - loss: 8.8133 - accuracy: 0.7272
Epoch 2: saving model to ../reports/2022-11-15/test/checkpoints/02.hdf5
100/100 [==============================] - 10s 96ms/step - loss: 8.8133 - accuracy: 0.7272
Epoch 3/5
100/100 [====